# Recurrent Neural Networks for Text Classification

## Learning Objectives:
1. Understand how a recurrent neural network differs from a feed forward network
2. Learn to implement RNNs in Keras
3. Explore word embeddings for natural language processing
4. Use NLP to classify documents using word embeddings

What is a Recurrent Neural Network?

![RNN Image](https://www.researchgate.net/profile/Ali-Daud/publication/330920746/figure/fig2/AS:741304577318912@1553752192108/Folded-and-unfolded-regular-recurrent-neural-networks-with-single-hidden-recurrent-layer.png)

Source: https://www.researchgate.net/figure/Recurrent-neural-networks-structure_fig4_326884617

In a nutshell, an RNN takes in X<sub>t</sub> and outputs 2 things:  

1. y<sub>t</sub> The output.
2. W<sub>t</sub> a value that is passed back into the layer and combined with X<sub>t+1</sub> to produce:
  1. y<sub>t+t</sub>
  2. W<sub>t+1</sub>

...and so on until the end of the sequence of the input.

It is a recurring process that acts on each feature of an input **In Sequence**.

The W value allows the model to pass information from previous X<sub>t-1</sub> inputs to affect how the model processes the X<sub>t<sub> input.

You can think of this as a single node that is calling itself over and over (most accurate) or as a series of identical nodes being called in sequence (easier, but less accurate conceptualization).

The values passed to the next layer in the model could consist of all of the Y values or just the final output of node after processing all of the inputs.  This is a hyperparameter you can tune in your model development.  Y output sequences can also make up final model outputs where a full sequence is desired, not just a classification.  Translation models would be an example of this impelementation.  Forecasting a time series would be another.

There are many flavors of RNNs and what sets them apart is how the W is formed and what information is passed between recurrences.  

*Note* Just a like a densely connected layer, information is potentially passed through multiple of these recurring nodes in parallel for each layer in the model.

![multi-node RNN](https://gblobscdn.gitbook.com/assets%2F-LvBP1svpACTB1R1x_U4%2F-LwEQnQw8wHRB6_2zYtG%2F-LwEZT8zd07mLDuaQZwy%2Fimage.png?alt=media&token=93a3c3e2-e32b-4fec-baf5-5e6b092920c4)

Source: https://docs.paperspace.com/machine-learning/wiki/recurrent-neural-network-rnn

# Why RNNs?

While a feed forward network of dense layers processes each feature independently, an RNN treats them as a sequence in which the order of features is important.  This makes it ideal for:

1. Time series data
2. Audio data
3. Natural language
4. Anything else where the order of features carries important information.

# Memory Cells:

LSTMs, or Long-Short Term Memory RNNS and GRUs, Gated Recurrent Units, pass an additional value with the W, a memory cell.  While the W is fully dependent on the X and the previous W<sub>t-1</sub> values, the memory cell has potentially persistent values through multiple loops.  This allows the model have a more persistent memory of inputs from many steps previous to X<sub>t</sub>.  

For example, a GRU might learn to remember the tense of a sentence or the gender of a character.

![RNNs, LSTMs, and GRUs](https://slidetodoc.com/presentation_image_h/8a65d4524f55935cb441b783479395ed/image-33.jpg)

Source: https://slidetodoc.com/recurrent-neural-networks-deeplearning-ai-gated-recurrent-unit/

# What are the trainable Parameters:

The trainable parameters and activation functions in an RNN are the various weights and functions are inside each individual parallel node.  There are, potentially, more than one weight depending on the flavor of RNN.  However, the same weights are used for each feature in the sequence for a given node.

This is different than a dense node where each feature has a separate weight.  One potential benefit of this is that certain configurations of RNNs can process **variable** length inputs, which feed forward networks cannot.

#Further Reading

Here is a wonderful article by Michael Phi with animations that does a great job explaining further:  https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

# LSTMs in Keras

Keras has a pretty simple implementation of RNN layers.  Below I will demonstrate how to create an LSTM model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense, Dropout, InputLayer, LSTM, \
Flatten, GlobalMaxPool1D
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import Constant


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [ ]:

def plot_history(history):
    """
    Given a model history will plot the model training history and 
    return the last scores for each loss and metric in the  model.
    Returns None.
    """
    hist = pd.DataFrame(history.history)
    fig = plt.figure(figsize = (10,5))
    ax = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    loss = hist.columns[hist.columns.str.endswith('loss')]
    accuracy = hist.columns[hist.columns.str.endswith('accuracy')]
    hist[loss].plot(title='Loss', ax=ax)
    hist[accuracy].plot(title='accuracy', ax=ax2)
    plt.show()
    
    for l in loss:
        print(f'final {l}: {hist[l].iloc[-1]}')
    for r in accuracy:
        print(f'final {r}: {hist[r].iloc[-1]}')
    plt.show()

## The Data:

We are going to return to our horror author data.  This is a collection of sentences from horror novels written by Edgar Allen Poe, H.P. Lovecraft, or Mary Shelley.  The task is to classify each sentency by the author who wrote it.

In [ ]:
#load the data 
spooky = pd.read_csv('https://raw.githubusercontent.com/learn-co-students/011121-pt-ds/main/phase_4/nlp/horror_writers.csv', 
                     usecols=['text','author'])
spooky.head()

In [ ]:
X = spooky.text
y = spooky.author

#One hot encode labels for multiclass classification in Keras
# lb = LabelBinarizer()
# y= lb.fit_transform(y)
y = LabelEncoder().fit_transform(y)
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=123)
X_val, X_holdout, y_val, y_holdout = train_test_split(X_test, y_test, test_size = .5, random_state=123)

## TextVectorization Layer

We are going to use a special Keras layer called TextVectorization that will process our input for us and transform into a sequences of integers to pass to our LSTM layer.  Layers like this can make models more like pipelines.  Notice that we only fit the vectorizer layer on the training set to avoid data leakage.

In [ ]:
#max length of input.  
#Will truncate inputs longer and add 0s to the end of sequences that are longer.
output_sequence_length = spooky.text.str.len().max()

vectorizer = TextVectorization(output_sequence_length=output_sequence_length,
                                   )
vectorizer.adapt(X_train.to_numpy())
vocab_len = vectorizer.vocabulary_size()

In [ ]:
def create_LSTM():
  model = Sequential()

  #Define your input layer for one feature (the whole string) and the dtype.
  #If you don't let Keras know to expect a string, it will assume it's looking
  #for a float and you'll get an error.
  model.add(InputLayer(input_shape=(1,), dtype=tf.string))

  #Fitted TextVectorization layer
  model.add(vectorizer)

  #Untrained Embedding Layer, embedding dimensions of 300
  #We'll talk about this layer more a little farther down
  model.add(Embedding(vocab_len, 300, input_length=output_sequence_length))
  
  #The recurrent LSTM layer.  We have it return all of its Y outputs for 
  #each cycle of each layer.
  model.add(LSTM(50, return_sequences=True, dropout=0.5, 
                 kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)))
  #The below below layer returns the highest activation of each LSTM node
  #for each input and pass them to the dense layer.
  model.add(GlobalMaxPool1D())
  
  model.add(Dense(50, activation='relu', 
                  kernel_regularizer = regularizers.l1_l2(l1=1e-4, l2=1e-3)))  
  model.add(Dropout(0.5))
  
  model.add(Dense(50, activation='relu', 
                  kernel_regularizer = regularizers.l1_l2(l1=1e-4, l2=1e-3)))  
  model.add(Dropout(0.5))
  
  #Add an output layer.  3 nodes for 3 classes and a softmax activation
  model.add(Dense(3, activation='softmax'))

  optimizer = optimizers.Adam(learning_rate=.01)
  model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

  return model

In [ ]:
%%time
model = create_LSTM()

model.summary()

history = model.fit(X_train,
                    y_train,
                    validation_data = (X_val, y_val),
                    epochs = 10,
                    batch_size = 128)


In [ ]:

self_trained_score = model.evaluate(X_test, y_test)
print(f'Accraucy on Test Set {self_trained_score[1]}, Loss: {self_trained_score[0]}')
plot_history(history)

# Word Embeddings

You have have noticed that we used tf.keras.layers.Embedding() after the text vectorization layer.  This is an amazingly awesome way of encoding words.  It transforms each word from an integer (from the vectorizer layer) to a vector.  What does it mean for a word to be a vector?

It means the representation of the word can contain arbitrary amounts of information about that word.  Each dimension of the space that the word vector exists in contains information about that word.

We can determine relationships between words and define similarity.

These embeddings are discovered using unsupervised learning on large datasets of text.  Several weights for embedding models are available that have been trained for thousands of hours on hundreds of millions of texts (All of wikipedia, or instance).  

The results are a library of {word:vector} pairs with each dimension of the vector representing a relationship between words learned from the corpus the model was trained on.

![Word embedding space](https://developers.google.com/machine-learning/crash-course/images/linear-relationships.svg)

Source: https://developers.google.com/machine-learning/crash-course/embeddings/translating-to-a-lower-dimensional-space


In [ ]:
from sklearn.decomposition import PCA


import gensim.downloader as api
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
word_vectors = api.load("glove-wiki-gigaword-100")

We can look up vectors like a dictionary

In [ ]:
word_vectors['coffee']

We can also use the gensim model to find relationships between words. For instance:

1. Similarity

In [ ]:
word_vectors.most_similar('coffee')

2. Analogies

In [ ]:
def analogy(x1, x2, y1):
    result = word_vectors.most_similar(positive=[y1, x2], negative=[x1])
    return result

In [ ]:
analogy('japan','japanese','america')

In [ ]:
analogy('man','king','woman')

# Bias in Word Embeddings
These relationships are learned from texts from the real world and carry the associations present in them.  We need to be aware of introducing bias into models that may affect people's lives.

In [ ]:
analogy('man','doctor','woman')

## Graphical Representation

In [ ]:
def display_pca_scatterplot(model, words=None, sample=0):
    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
        
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

In [ ]:
display_pca_scatterplot(word_vectors, 
                        ['coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 'water',
                         'spaghetti', 'borscht', 'hamburger', 'pizza', 'falafel', 'sushi', 'meatballs',
                         'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard',
                         'frog', 'toad', 'monkey', 'ape', 'kangaroo', 'wombat', 'wolf',
                         'france', 'germany', 'hungary', 'luxembourg', 'australia', 'fiji', 'china',
                         'homework', 'assignment', 'problem', 'exam', 'test', 'class',
                         'school', 'college', 'university', 'institute'])

# SpaCy

Gensim is great because it lets us easily explore word embeddings.  SpaCy is great because it loads word embeddings much faster, allowing us to use larger embedding models, and has some great functionality for NLP, including parts of speech, processing, and more.

In [ ]:
import spacy

# download and import the large english model.
!python -m spacy download en_core_web_lg
import en_core_web_lg
# nlp will be our word embedding model
nlp = en_core_web_lg.load()


In our last model, we allowed our embedding layer to learn its own embedding vectors from the texts in our training corpus.  But what if we used transfer learning to instead load the weights from a pretrained word embedding model?

# Creating Word Embedding Weights

We have to create our own matrix of weights for our embedding layer.  We do this by using our vectorizer to create an index for a matrix, and filling in each row of the matrix with the word embedding provided by our pretrained word embedding model.  The result is a (vocab, embedding_dimension) shaped numpy array that our LSTM layer can use as a lookup table to find the correct embedding for each word in our text samples.

Finally, we freeze those weights to reduce computation time.

## LSTM
As you recall, each node of the LSTM layer processes each word in each document in order, using the word embedding from the Embedding layer as the value for that word.  LSTMs 'remember' information from previous items in the sequence, words in the sentence in this case, and carry that information forward to help process later items in the sequence.


In [ ]:

#generate the embedding matrix
vocab = vectorizer.get_vocabulary()
num_tokens = len(vocab)
embedding_dim = len(nlp('The').vector)

#Generate the embedding weights usign the SpaCy word vectors for the vocabulary
#in our training corpus.  (This may take awhile)
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i, word in enumerate(vocab):
    embedding_matrix[i] = nlp(word).vector

#Load the embedding matrix as the weights matrix for the embedding layer and set trainable to False
Embedding_layer=Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False)

In [ ]:
def create_pretrain_LSTM():
  model = Sequential()

  #Define your input layer for one feature (the whole string) and the dtype.
  #If you don't let Keras know to expect a string, it will assume it's looking
  #for a float and you'll get an error.
  model.add(InputLayer(input_shape=(1,), dtype=tf.string))

  #Fitted TextVectorization layer
  model.add(vectorizer)

  #Untrained Embedding Layer, embedding dimensions of 300
  #We'll talk about this layer more a little farther down
  model.add(Embedding_layer)
  
  #The recurrent LSTM layer.  We have it return all of its Y outputs for 
  #each cycle of each layer.
  model.add(LSTM(50, return_sequences=True, dropout=0.5, 
                 kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3)))
  #The below below layer returns the highest activation of each LSTM node
  #for each input and pass them to the dense layer.
  model.add(GlobalMaxPool1D())
  
  model.add(Dense(50, activation='relu', 
                  kernel_regularizer = regularizers.l1_l2(l1=1e-4, l2=1e-3)))  
  model.add(Dropout(0.5))
  
  model.add(Dense(50, activation='relu', 
                  kernel_regularizer = regularizers.l1_l2(l1=1e-4, l2=1e-3)))  
  model.add(Dropout(0.5))
  
  #Add an output layer.  3 nodes for 3 classes and a softmax activation
  model.add(Dense(3, activation='softmax'))

  optimizer = optimizers.Adam(learning_rate=.01)
  model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

  return model

In [ ]:
%%time
tf.keras.backend.clear_session()

model = create_pretrain_LSTM()

model.summary()

history = model.fit(X_train,
                    y_train,
                    validation_data = (X_val, y_val),
                    epochs = 10,
                    batch_size = 128)

pretrained_score = model.evaluate(X_test, y_test)



In [ ]:

plot_history(history)
print(f'Accuracy of Self-trained Embedding model: {self_trained_score[1]}, Loss: {self_trained_score[0]}')
print(f'Accuracy of Pretrained Embedding model: {pretrained_score[1]}, Loss: {pretrained_score[0]}')

# Conclusion

In this lesson you learned about recurrent neural networks, or RNNs, how to implement them in Keras, and how to use transfer learning to improve a natural language processing modeling task.

I hope you feel comfortable and confident to use these powerful tools in your own work.

# References:

[Pretrained Word Embeddings using SpaCy and Keras TextVectorization](https://towardsdatascience.com/pretrained-word-embeddings-using-spacy-and-keras-textvectorization-ef75ecd56360)

[Multi-Class Text Classification with LSTM](https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17)